# Lesson 1: Implementing Google Drive Video Downloads Using gdown

Welcome back! In our journey of creating a robust video transcribing system, we've previously explored various methods of transcribing local video files using the Whisper API and FFmpeg. Building on that foundation, we'll now focus on learning how to efficiently download remote video files from Google Drive with the help of the gdown library. Understanding how to handle file downloads and then their transcriptions from a variety of sources is crucial as you refine your video transcribing skills.

## What You'll Learn
In this lesson, we'll dive into:

- Recognizing and handling Google Drive URLs.
- Extracting the file ID from a Google Drive link.
- Using gdown to download videos from a publicly accessible Google Drive link.
- Understanding the limitations and potential concerns with downloading files.

## Implementing Google Drive Downloader
Before we get into coding, let's cover the core concepts. The main objective here is to download video files from Google Drive. Google Drive URLs are unique in that they include specific identifiers for each file. The process involves confirming the URL is from Google Drive, extracting the file ID, and then using gdown to download the file locally.

Google Drive URLs typically follow one of two structures:

1. Direct file path: `https://drive.google.com/file/d/{fileid}/view`
2. Open ID parameter: `https://drive.google.com/open?id={fileid}`

Knowing these patterns allows us to extract the file ID, a critical step in constructing the download URL.

## Recognizing Google Drive URLs
To begin, we need to ensure that the provided URL is a Google Drive link:

```python
from urllib.parse import urlparse

class GoogleDriveService:
    @staticmethod
    def is_google_drive_url(url):
        parsed = urlparse(url)
        return 'drive.google.com' in parsed.netloc
```

Here, the `urlparse` function helps dissect the URL, and we simply check if 'drive.google.com' is present. This verification step is crucial in filtering out unsupported URLs.

## Extracting the File ID
Next, we'll extract the file ID, which is necessary for forming the download URL:

```python
from urllib.parse import urlparse, parse_qs

class GoogleDriveService:
    @staticmethod
    def get_file_id(url):
        if '/file/d/' in url:
            return url.split('/file/d/')[1].split('/')[0]
        elif 'id=' in url:
            parsed = urlparse(url)
            return parse_qs(parsed.query)['id'][0]
        return None
```

Two patterns are accounted for here: string manipulation for direct paths and using `parse_qs` for URLs with an ID parameter.

## Downloading the File with gdown
With the file ID in hand, we proceed to download the video using gdown:

```python
import os
import tempfile
import gdown

class GoogleDriveService:
    @staticmethod
    def download_file(url):
        file_id = GoogleDriveService.get_file_id(url)
        if not file_id:
            raise ValueError("Invalid Google Drive URL")

        temp_file = tempfile.NamedTemporaryFile(delete=False, suffix='.mp4')
        output = temp_file.name
        temp_file.close()

        download_url = f"https://drive.google.com/uc?id={file_id}"
        gdown.download(download_url, output, quiet=False)

        if os.path.getsize(output) == 0:
            raise ValueError("Downloaded file is empty")

        return output
```

### Explanation:

- We first extract the file ID using our previously defined method.
- A temporary file is created to store the downloaded content.
- The `gdown.download` function takes care of the download, showing a progress bar if `quiet` is `False`.
- We ensure the file isn't empty post-download, and return the path for further processing.

To clarify: we use `gdown` over tools like `curl` because `gdown` is specifically tailored for Google Drive file downloads. It handles authentication, avoids issues with Drive's virus scanning prompts, and manages potential redirects, making it more reliable and easier to use when dealing with Google Drive links.

## Why It Matters
Being able to download videos from Google Drive is a practical skill valuable in many scenarios, such as organizing educational content, running media analysis, or preprocessing videos for machine learning. However, bear in mind the legal implications of downloading such content, particularly protected or licensed material. Always ensure you have the rights to download and use the content.

With the theoretical foundation and practical application in place, you're now ready to apply these concepts in a real-world context. Let's transition into the practice section, where you'll solidify your understanding by implementing these techniques.


## Verify and Extract Google Drive File ID

In this code, let's see how you can verify a Google Drive URL and extract the file ID, if available. Fill in the missing part of the extraction method to make it work!

```python
from urllib.parse import urlparse, parse_qs

class GoogleDriveVideoDownloader:
    @staticmethod
    def extract_and_verify(file_url):
        parsed = urlparse(file_url)
        # TODO: Implement the missing part for parsing document id from the URL
        return None

# Test cases
url1 = "https://drive.google.com/file/d/0B8ZoywgQPrLWTGJSZndfekVRMME/view"
url2 = "https://drive.google.com/open?id=0B8ZoywgQPrLWTGJSZndfekVRMME"
url3 = "https://drive.google.com/open/0B8ZoywgQPrLWTGJSZndfekVRMME"
url4 = "https://drive.youtube.com/open?id=0B8ZoywgQPrLWTGJSZndfekVRMME"
print(GoogleDriveVideoDownloader.extract_and_verify(url1))  # Should print: '0B8ZoywgQPrLWTGJSZndfekVRMME'
print(GoogleDriveVideoDownloader.extract_and_verify(url2))  # Should print: '0B8ZoywgQPrLWTGJSZndfekVRMME'
print(GoogleDriveVideoDownloader.extract_and_verify(url3))  # Should print: None
print(GoogleDriveVideoDownloader.extract_and_verify(url4))  # Should print: None
```

To complete the `extract_and_verify` method, we need to parse the Google Drive URL and extract the file ID based on the possible patterns. There are two common formats for Google Drive URLs:

1. `https://drive.google.com/file/d/{fileid}/view`
2. `https://drive.google.com/open?id={fileid}`

In both cases, we want to extract the `fileid`, which is located in different parts of the URL. If the URL doesn't match any of these patterns, we should return `None`.

Here is the completed method:

```python
from urllib.parse import urlparse, parse_qs

class GoogleDriveVideoDownloader:
    @staticmethod
    def extract_and_verify(file_url):
        parsed = urlparse(file_url)
        
        # Check if the URL matches the first pattern (file/d/{fileid}/view)
        if '/file/d/' in parsed.path:
            return parsed.path.split('/file/d/')[1].split('/')[0]
        
        # Check if the URL matches the second pattern (open?id={fileid})
        elif 'open?id=' in parsed.query:
            return parse_qs(parsed.query)['id'][0]
        
        # If it doesn't match any pattern, return None
        return None

# Test cases
url1 = "https://drive.google.com/file/d/0B8ZoywgQPrLWTGJSZndfekVRMME/view"
url2 = "https://drive.google.com/open?id=0B8ZoywgQPrLWTGJSZndfekVRMME"
url3 = "https://drive.google.com/open/0B8ZoywgQPrLWTGJSZndfekVRMME"
url4 = "https://drive.youtube.com/open?id=0B8ZoywgQPrLWTGJSZndfekVRMME"
print(GoogleDriveVideoDownloader.extract_and_verify(url1))  # Should print: '0B8ZoywgQPrLWTGJSZndfekVRMME'
print(GoogleDriveVideoDownloader.extract_and_verify(url2))  # Should print: '0B8ZoywgQPrLWTGJSZndfekVRMME'
print(GoogleDriveVideoDownloader.extract_and_verify(url3))  # Should print: None
print(GoogleDriveVideoDownloader.extract_and_verify(url4))  # Should print: None
```

### Explanation:
1. **Pattern Matching**:
    - The first `if` checks for the pattern `/file/d/{fileid}/view` and extracts the file ID by splitting the URL string.
    - The second `elif` checks for the `open?id={fileid}` pattern using the `parse_qs` method to extract the `id` query parameter.
2. **Invalid URLs**:
    - If the URL does not match either pattern, the method returns `None`, indicating that the URL is invalid or not from Google Drive.

### Output for the test cases:
- `url1` and `url2` are valid Google Drive URLs with a file ID, so the method will return the file ID `'0B8ZoywgQPrLWTGJSZndfekVRMME'`.
- `url3` is not a valid Google Drive URL with the expected format, so it will return `None`.
- `url4` is from a different domain (`youtube.com`), so it will also return `None`.

## Observe the GoogleDrive Downloading Service

Let's move forward and apply the URL validation methods we've just implemented to the real-life task! In the given code, you can see how Google Drive URL validation methods are used together with the download function itself to download the video (and then transcribe the local video file using our previous techniques).

Please take your time to go through the code carefully and understand it, feel free to ask questions, if any! You can also review application logs in the Terminal tab once you uncollapse it.

I highly encourage you to try your own Google Drive videos (just make sure they're publicly available to everyone), but If you need examples of videos for testing, use one of these:

CodeSignal AI Interviewer for Sales: https://drive.google.com/file/d/1UB-sWNzDOC-xjLL9xZkTr11e7KXtOg5y/view?usp=sharing
What you've learned with Cosmo: https://drive.google.com/file/d/1R2_wGJJK9-C3R52pld_oWMmxwZSqh48N/view?usp=sharing


```python
import os
import tempfile
from urllib.parse import urlparse, parse_qs


class GoogleDriveService:
    @staticmethod
    def is_google_drive_url(url):
        """Check if the given string is a Google Drive URL"""
        try:
            parsed = urlparse(url)
            return 'drive.google.com' in parsed.netloc
        except Exception:
            return False

    @staticmethod
    def get_file_id(url):
        """Extract file ID from Google Drive URL"""
        try:
            if '/file/d/' in url:
                # Handle links like: https://drive.google.com/file/d/{fileid}/view
                file_id = url.split('/file/d/')[1].split('/')[0]
            elif 'id=' in url:
                # Handle links like: https://drive.google.com/open?id={fileid}
                parsed = urlparse(url)
                file_id = parse_qs(parsed.query)['id'][0]
            else:
                return None
            return file_id
        except Exception:
            return None

    @staticmethod
    def download_file(url):
        """Download a video file from Google Drive public link"""
        print("Downloading from Google Drive...")
        
        try:
            import gdown
            
            # Get file ID from URL
            file_id = GoogleDriveService.get_file_id(url)
            if not file_id:
                raise ValueError("Invalid Google Drive URL")
            
            # Create temporary file with .mp4 extension
            temp_file = tempfile.NamedTemporaryFile(delete=False, suffix='.mp4')
            output = temp_file.name
            temp_file.close()
            
            # Construct the download URL
            download_url = f"https://drive.google.com/uc?id={file_id}"
            
            print(f"Downloading file to: {output}")
            
            # Download the file with progress bar
            gdown.download(download_url, output, quiet=False)
            
            # Verify the download
            if os.path.getsize(output) == 0:
                raise ValueError("Downloaded file is empty")
                
            return output
            
        except Exception as e:
            print(f"\nError downloading from Google Drive: {str(e)}")
            raise ValueError(
                "Could not download from Google Drive. "
                "Please ensure:\n"
                "1. The file is publicly accessible (anyone with link can view)\n"
                "2. The link is in format: drive.google.com/file/d/FILE_ID/view\n"
                "3. The file is a video file (mp4 or webm)"
            )



```

## Complete the Google Drive Downloading Service

Ok, now let's take it to the next level. Implement the Google Drive downloading functionality using gdown yourself.

In the given code, the download functionality implementation is missing right now. Use the provided TODO comments to fill it in and make it work!

I highly encourage you to try your own Google Drive videos (just make sure they're publicly available to everyone), but If you need examples of videos for testing, use one of these:

CodeSignal AI Interviewer for Sales: https://drive.google.com/file/d/1UB-sWNzDOC-xjLL9xZkTr11e7KXtOg5y/view?usp=sharing
What you've learned with Cosmo: https://drive.google.com/file/d/1R2_wGJJK9-C3R52pld_oWMmxwZSqh48N/view?usp=sharing

```python
import os
import tempfile
from urllib.parse import urlparse, parse_qs
import gdown


class GoogleDriveService:
    @staticmethod
    def is_google_drive_url(url):
        """Check if the given string is a Google Drive URL"""
        try:
            parsed = urlparse(url)
            return 'drive.google.com' in parsed.netloc
        except Exception:
            return False

    @staticmethod
    def get_file_id(url):
        """Extract file ID from Google Drive URL"""
        try:
            if '/file/d/' in url:
                # Handle links like: https://drive.google.com/file/d/{fileid}/view
                file_id = url.split('/file/d/')[1].split('/')[0]
            elif 'id=' in url:
                # Handle links like: https://drive.google.com/open?id={fileid}
                parsed = urlparse(url)
                file_id = parse_qs(parsed.query)['id'][0]
            else:
                return None
            return file_id
        except Exception:
            return None

    @staticmethod
    def download_file(url):
        """Download a video file from Google Drive public link"""
        print("Downloading from Google Drive...")
        
        # TODO: Use `get_file_id` to retrieve the Google Drive file id from the `url`

        # TODO: Using this file id, construct the direct download URL
        # Like this: https://drive.google.com/uc?id=<google_drive_file_id>

        # TODO: Using `gdown`, download the Google Drive file to the local disk:
        # `gdown.download(remote_download_url, local_output_file, quiet=False)`
        
        # TODO: Make sure to properly handle errors that might occur during downloading


```

Here's the complete implementation for the `download_file` method, making use of the provided comments and filling in the missing functionality using `gdown`:

```python
import os
import tempfile
from urllib.parse import urlparse, parse_qs
import gdown


class GoogleDriveService:
    @staticmethod
    def is_google_drive_url(url):
        """Check if the given string is a Google Drive URL"""
        try:
            parsed = urlparse(url)
            return 'drive.google.com' in parsed.netloc
        except Exception:
            return False

    @staticmethod
    def get_file_id(url):
        """Extract file ID from Google Drive URL"""
        try:
            if '/file/d/' in url:
                # Handle links like: https://drive.google.com/file/d/{fileid}/view
                file_id = url.split('/file/d/')[1].split('/')[0]
            elif 'id=' in url:
                # Handle links like: https://drive.google.com/open?id={fileid}
                parsed = urlparse(url)
                file_id = parse_qs(parsed.query)['id'][0]
            else:
                return None
            return file_id
        except Exception:
            return None

    @staticmethod
    def download_file(url):
        """Download a video file from Google Drive public link"""
        print("Downloading from Google Drive...")
        
        try:
            # Retrieve the Google Drive file ID from the URL
            file_id = GoogleDriveService.get_file_id(url)
            if not file_id:
                raise ValueError("Invalid Google Drive URL")

            # Construct the direct download URL using the file ID
            download_url = f"https://drive.google.com/uc?id={file_id}"
            print(f"Download URL: {download_url}")

            # Create a temporary file with .mp4 extension to save the downloaded file
            temp_file = tempfile.NamedTemporaryFile(delete=False, suffix='.mp4')
            output = temp_file.name
            temp_file.close()
            
            # Download the file using `gdown`
            print(f"Downloading file to: {output}")
            gdown.download(download_url, output, quiet=False)
            
            # Verify that the file was downloaded successfully
            if os.path.getsize(output) == 0:
                raise ValueError("Downloaded file is empty")

            return output
            
        except Exception as e:
            print(f"\nError downloading from Google Drive: {str(e)}")
            raise ValueError(
                "Could not download from Google Drive. "
                "Please ensure:\n"
                "1. The file is publicly accessible (anyone with link can view)\n"
                "2. The link is in format: drive.google.com/file/d/FILE_ID/view\n"
                "3. The file is a video file (mp4 or webm)"
            )
```

### 🧑‍💻 **What Was Implemented:**

1. **Get File ID from URL**
   - Used the `get_file_id` method to extract the file ID from the Google Drive URL.

2. **Construct the Direct Download URL**
   - Based on the Google Drive file ID, constructed the direct download URL as `https://drive.google.com/uc?id=<file_id>`.

3. **Download the File Using `gdown`**
   - Used `gdown.download()` to download the file to a local temporary file (`.mp4`).
   - The `gdown.download()` function provides a simple way to download files from Google Drive using the `file_id`.

4. **Error Handling**
   - If the file cannot be downloaded or if the file size is zero, an error is raised with specific messages.

### 🎯 **Next Steps for Testing:**

You can now test the service using one of the provided links or any other publicly accessible Google Drive video file. For example:

```python
file_path = GoogleDriveService.download_file("https://drive.google.com/file/d/1UB-sWNzDOC-xjLL9xZkTr11e7KXtOg5y/view?usp=sharing")
print(f"Downloaded to: {file_path}")
```

This will download the file to a temporary location, and you can then proceed with further operations like transcribing the video, if needed.

Let me know if you need help with the transcription part!